In [130]:
%load_ext autoreload
%autoreload 2

import pickle
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

import sys
sys.path.append("../")
from src import soporte_eda_y_outliers as s_eda, soporte_encoding as s_encoding, soporte_scaling as s_scaling
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

pd.set_option('display.max_columns', None)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Vamos a usar el mismo tratamiento de nuelos que usamos con el modelo 1, anque los datos estén en dólares
. 


In [131]:
df = pd.read_pickle("../datos_dfs\pickle1_nonulos.pkl") 

Vamos a hacer un IFO pues coge los valores más aislados. Con la mínima contaminación posible, a ver qué sale

In [132]:
outliers_multivariados = s_eda.GestionOutliersMultivariados(df)
df_outliers_ifo = outliers_multivariados.detectar_outliers_if(contaminacion=[0.01])

Los outliers que tenemos son de gente que o cobra mucho o cobra muuuuy poco. Parecen errores porque hay un "Research Director" divorciado que cobra 240 dólares. Eliminamos todos.

In [133]:
prueba, df_sin_outliers = outliers_multivariados.eliminar_outliers_abs(df_outliers_ifo)
prueba

se han detectado 15 columnas con outliers absolutos


,JobInvolvement,PerformanceRating,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,StockOptionLevel,TrainingTimesLastYear,YearsWithCurrManager,EnvironmentSatisfaction_iterative,JobSatisfaction_iterative,WorkLifeBalance_iterative,NumCompaniesWorked_iterative,TotalWorkingYears_iterative,outliers_ifo_0.01_1000,outliers_ifo_0.01_2000
105,1,4,1,Travel_Rarely,Human Resources,28,2,Human Resources,Female,5,Manager,Single,667.32,1,3,0,4,3,3,1,1,-1,-1
182,4,4,1,Travel_Frequently,Sales,17,1,Marketing,Male,1,Research Director,Divorced,240.12,1,5,2,2,1,3,9,11,-1,-1
235,4,4,0,Travel_Rarely,Sales,29,4,Marketing,Male,2,Sales Executive,Single,234.12,3,4,2,1,3,4,4,10,-1,-1
286,2,4,0,Travel_Frequently,Research & Development,27,3,Life Sciences,Male,1,Research Scientist,Divorced,652.92,1,6,13,4,3,2,9,28,-1,-1
360,4,3,0,Travel_Rarely,Research & Development,17,3,Medical,Male,1,Laboratory Technician,Single,2391.12,3,5,1,1,1,2,7,6,-1,-1
521,3,4,1,Travel_Rarely,Sales,23,3,Medical,Female,2,Research Scientist,Married,2312.64,0,2,7,1,3,4,9,18,-1,-1
578,1,3,1,Travel_Rarely,Research & Development,6,3,Life Sciences,Male,1,Sales Executive,Married,269.28,3,5,10,4,1,1,1,11,-1,-1
717,4,3,1,Travel_Rarely,Research & Development,19,3,Technical Degree,Male,1,Human Resources,Single,561.60,3,0,8,1,3,2,6,24,-1,-1
799,3,3,1,Travel_Frequently,Research & Development,28,3,Medical,Female,5,Research Scientist,Single,532.08,3,1,0,1,2,1,2,7,-1,-1
820,2,3,0,Travel_Rarely,Sales,26,1,Marketing,Male,1,Sales Representative,Married,1291.32,1,5,17,4,1,4,3,24,-1,-1


# ENCODING

A la hora de encodear aplicaremos one-hot encoding a MaritalStatus por su importancia

In [134]:
sep = s_eda.Visualizador(df)
df_num,df_cat = sep.separar_dataframes()

In [135]:
for col in df.columns:
    if col in df_num.columns:
        df.rename(columns={col: col + "_num"},inplace= True)
    elif col in df_cat.columns:
        df.rename(columns={col: col + "_cat"},inplace= True)  

In [136]:
sep = s_eda.Visualizador(df)
df_num,df_cat = sep.separar_dataframes()

In [137]:
df.columns

Index(['JobInvolvement_num', 'PerformanceRating_num', 'Attrition_num',
       'BusinessTravel_cat', 'Department_cat', 'DistanceFromHome_num',
       'Education_num', 'EducationField_cat', 'Gender_cat', 'JobLevel_num',
       'JobRole_cat', 'MaritalStatus_cat', 'MonthlyIncome_num',
       'StockOptionLevel_num', 'TrainingTimesLastYear_num',
       'YearsWithCurrManager_num', 'EnvironmentSatisfaction_iterative_num',
       'JobSatisfaction_iterative_num', 'WorkLifeBalance_iterative_num',
       'NumCompaniesWorked_iterative_num', 'TotalWorkingYears_iterative_num'],
      dtype='object')

In [138]:
df.rename(columns={"Attrition_num": "Attrition"},inplace= True)

In [139]:
lista_cats = df_cat.columns
tests = s_encoding.TestEstadisticos(df,"Attrition",lista_cats)
dic = tests.detectar_orden_binaria()

Esta es la tabla de contingencia para BusinessTravel_cat, con la variable respuesta Attrition
Attrition             0    1
BusinessTravel_cat          
Non-Travel          144   12
Travel_Frequently   217   76
Travel_Rarely       931  163
La variable categórica TIENE ORDEN
-----
Esta es la tabla de contingencia para Department_cat, con la variable respuesta Attrition
Attrition                 0    1
Department_cat                  
Human Resources          44   21
Research & Development  852  160
Sales                   396   70
La variable categórica TIENE ORDEN
-----
Esta es la tabla de contingencia para EducationField_cat, con la variable respuesta Attrition
Attrition             0    1
EducationField_cat          
Human Resources      16   12
Life Sciences       534  107
Marketing           137   26
Medical             401   81
Other                79   10
Technical Degree    125   15
La variable categórica TIENE ORDEN
-----
Esta es la tabla de contingencia para Gender_cat, con la 

In [140]:
dic_encoding = {'onehot': ['Gender_cat', 'JobRole_cat','MaritalStatus_cat'],
 'target': ['BusinessTravel_cat', 'Department_cat', "EducationField_cat" ]}

In [141]:
df.select_dtypes(include= "O").columns

Index(['BusinessTravel_cat', 'Department_cat', 'EducationField_cat',
       'Gender_cat', 'JobRole_cat', 'MaritalStatus_cat'],
      dtype='object')

In [142]:
encoding = s_encoding.Encoding(df,dic_encoding,"Attrition")
df_enco = encoding.one_hot_encoding()
encoding = s_encoding.Encoding(df_enco,dic_encoding,"Attrition")
df_enco = encoding.target_encoding()
df_enco.reset_index(drop=True)
df_enco.drop(columns = dic_encoding["onehot"],inplace= True)

In [143]:
df_enco.columns

Index(['JobInvolvement_num', 'PerformanceRating_num', 'Attrition',
       'BusinessTravel_cat', 'Department_cat', 'DistanceFromHome_num',
       'Education_num', 'EducationField_cat', 'JobLevel_num',
       'MonthlyIncome_num', 'StockOptionLevel_num',
       'TrainingTimesLastYear_num', 'YearsWithCurrManager_num',
       'EnvironmentSatisfaction_iterative_num',
       'JobSatisfaction_iterative_num', 'WorkLifeBalance_iterative_num',
       'NumCompaniesWorked_iterative_num', 'TotalWorkingYears_iterative_num',
       'Gender_cat_Female', 'Gender_cat_Male',
       'JobRole_cat_Healthcare Representative', 'JobRole_cat_Human Resources',
       'JobRole_cat_Laboratory Technician', 'JobRole_cat_Manager',
       'JobRole_cat_Manufacturing Director', 'JobRole_cat_Research Director',
       'JobRole_cat_Research Scientist', 'JobRole_cat_Sales Executive',
       'JobRole_cat_Sales Representative', 'MaritalStatus_cat_Divorced',
       'MaritalStatus_cat_Married', 'MaritalStatus_cat_Single'],
    

Aprovechamos para eliminar columnas que no aportan mucho en el modelo original, para evitar overfitting

In [144]:
df_enco.drop(columns= ['JobRole_cat_Research Scientist','JobRole_cat_Human Resources','JobRole_cat_Research Director','Gender_cat_Female','PerformanceRating_num','JobRole_cat_Sales Executive'],inplace= True)

In [145]:
df_enco.columns

Index(['JobInvolvement_num', 'Attrition', 'BusinessTravel_cat',
       'Department_cat', 'DistanceFromHome_num', 'Education_num',
       'EducationField_cat', 'JobLevel_num', 'MonthlyIncome_num',
       'StockOptionLevel_num', 'TrainingTimesLastYear_num',
       'YearsWithCurrManager_num', 'EnvironmentSatisfaction_iterative_num',
       'JobSatisfaction_iterative_num', 'WorkLifeBalance_iterative_num',
       'NumCompaniesWorked_iterative_num', 'TotalWorkingYears_iterative_num',
       'Gender_cat_Male', 'JobRole_cat_Healthcare Representative',
       'JobRole_cat_Laboratory Technician', 'JobRole_cat_Manager',
       'JobRole_cat_Manufacturing Director',
       'JobRole_cat_Sales Representative', 'MaritalStatus_cat_Divorced',
       'MaritalStatus_cat_Married', 'MaritalStatus_cat_Single'],
      dtype='object')

In [146]:
cols_sin_vr = ['JobInvolvement_num', 'BusinessTravel_cat',
       'Department_cat', 'DistanceFromHome_num', 'Education_num',
       'EducationField_cat', 'JobLevel_num', 'MonthlyIncome_num',
       'StockOptionLevel_num', 'TrainingTimesLastYear_num',
       'YearsWithCurrManager_num', 'EnvironmentSatisfaction_iterative_num',
       'JobSatisfaction_iterative_num', 'WorkLifeBalance_iterative_num',
       'NumCompaniesWorked_iterative_num', 'TotalWorkingYears_iterative_num',
       'Gender_cat_Male', 'JobRole_cat_Healthcare Representative',
       'JobRole_cat_Laboratory Technician', 'JobRole_cat_Manager',
       'JobRole_cat_Manufacturing Director',
       'JobRole_cat_Sales Representative', 'MaritalStatus_cat_Divorced',
       'MaritalStatus_cat_Married', 'MaritalStatus_cat_Single']

In [147]:
#Standard
escalador_standard = StandardScaler()

for columna in cols_sin_vr:
    datos_transf_stand = escalador_standard.fit_transform(df_enco[[columna]])  
    df_enco[f"{columna}_stand"] = datos_transf_stand

df_enco.drop(columns= cols_sin_vr,inplace= True)

In [151]:
lista_categoricas = []
for col in df_enco.columns:
    if "_cat" in col:
        lista_categoricas.append(col)




In [152]:
lista_categoricas

['BusinessTravel_cat_stand',
 'Department_cat_stand',
 'EducationField_cat_stand',
 'Gender_cat_Male_stand',
 'JobRole_cat_Healthcare Representative_stand',
 'JobRole_cat_Laboratory Technician_stand',
 'JobRole_cat_Manager_stand',
 'JobRole_cat_Manufacturing Director_stand',
 'JobRole_cat_Sales Representative_stand',
 'MaritalStatus_cat_Divorced_stand',
 'MaritalStatus_cat_Married_stand',
 'MaritalStatus_cat_Single_stand']

In [148]:
df_enco.to_pickle("../datos_dfs/pickle2_stand.pkl")